In [57]:
import os
import arcpy
from arcpy import env
from arcpy.sa import *
from arcpy.da import *
import pandas as pd

In [58]:
def zonstatis(in_table, fields, value_raster,out_table):
    """统计分区矢量内的栅格信息
    :param in_table: 矢量分区
    :param fields: 矢量字段
    :param value_raster: 栅格数据
    :param out_table: 输出文件"""
    ZonalStatisticsAsTable(in_zone_data=in_table,zone_field=fields,in_value_raster=value_raster,out_table=out_table,ignore_nodata='#',statistics_type='ALL')

In [59]:
# 矢量数据路径
polygon_path = r"C:\Users\Runker\Desktop\CNT_TRAIN\CNT_PRE.shp"
# 栅格数据路径
raster_path = r'D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb'

In [60]:
# 统计结果表格存放路径
temp_table_gdb_path = r'F:\cache_data\zone_ana\dy\zone_temp.gdb'

In [61]:
# 使用Delete_management函数删除数据库中的所有内容
try:
    arcpy.Delete_management(temp_table_gdb_path)
except:
    pass
# 再创建一个新的数据库
arcpy.management.CreateFileGDB(os.path.dirname(temp_table_gdb_path), "zone_temp", "CURRENT")

<Result 'F:\\cache_data\\zone_ana\\dy\\zone_temp.gdb'>

In [62]:
# 切换工作空间
env.workspace = raster_path

In [63]:
# 获取需要处理的栅格列表
raster_list = arcpy.ListRasters()

In [64]:
[_.name for _ in arcpy.ListFields(polygon_path)]

['FID', 'Shape', 'OBJECTID', 'DLBM', 'DLMC', 'Shape_Leng', 'Shape_Area', 'DZ']

In [65]:
print(raster_list)

['DEM', 'AnalyticalHillshading', 'Aspect', 'ChannelNetworkBaseLevel', 'ChannelNetworkDistance', 'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'PlanCurvature', 'ProfileCurvature', 'RelativeSlopePosition', 'Slope', 'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth', 'Contrast', 'Correlation', 'Dissimilarity', 'Entropy', 'Homogeneity', 'Mean', 'ndvi', 'PCA_0', 'PCA_1', 'SecondMoment', 'Variance', 'PRE', 'SRA', 'TMP', 'VAP', 'WIND', 'BIO', 'LON', 'LAT', 'PH', 'DZ', 'DL', 'MRRTF', 'MRVBF']


In [66]:
check_raster_list = ['DEM', 'PlanCurvature', 'ProfileCurvature', 'Slope','ndvi', 'PCA_0', 'MRRTF', 'MRVBF']

In [67]:

# 逐一统计分析
for one_raster in check_raster_list:
    print(one_raster)
    # 分区统计
    zonstatis(polygon_path,'OBJECTID',one_raster,os.path.join(temp_table_gdb_path,one_raster))

DEM
PlanCurvature
ProfileCurvature
Slope
ndvi
PCA_0
MRRTF
MRVBF


### 构造训练表


In [68]:
# 切换工作空间
env.workspace = temp_table_gdb_path

In [69]:
# 获取表格列表
table_list = arcpy.ListTables()

In [70]:
point_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(polygon_path,['OBJECTID']))

In [74]:
point_df.rename(columns={'OBJECTID':'OBJECTID_1'},inplace=True)

In [72]:
# 指定需要的信息
line_fields = 'OBJECTID_1'
df_list = []
for one_table in table_list:
    # 将表转换为pandas数据帧
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(one_table, "*"))  # 确保数据表中无空值
    # 提取统计字段的数据
    merged_df = df[[line_fields]+ list(df.columns[-10:])]
    # 在统计数据列名中添加表名
    merged_df.columns = [line_fields]+[one_table + '_' + col for col in merged_df.columns[-10:]]
    # 添加到列表
    df_list.append(merged_df)
# 初始合并数据帧
df_merged = df_list[0]
# 逐个合并其余的数据帧
for df in df_list[1:]:
    df_merged = pd.merge(df_merged, df, on=line_fields, how='outer')

In [73]:
df_merged

,OBJECTID_1,DEM_COUNT,DEM_AREA,DEM_MIN,DEM_MAX,DEM_RANGE,DEM_MEAN,DEM_STD,DEM_SUM,DEM_MEDIAN,...,MRVBF_COUNT,MRVBF_AREA,MRVBF_MIN,MRVBF_MAX,MRVBF_RANGE,MRVBF_MEAN,MRVBF_STD,MRVBF_SUM,MRVBF_MEDIAN,MRVBF_PCT90
0,6,27.0,675.0,1023.799988,1030.599976,6.799988,1027.181471,2.022101,27733.899719,1026.800049,...,27.0,675.0,0.009058,0.415623,0.406565,0.219172,0.118573,5.917642,0.226852,0.381662
1,7,40.0,1000.0,822.599976,834.400024,11.800049,826.194997,3.002248,33047.799866,825.600006,...,40.0,1000.0,0.001301,2.913924,2.912623,1.240179,1.043434,49.607156,1.272079,2.831860
2,8,212.0,5300.0,763.400024,771.000000,7.599976,765.955661,2.030737,162382.600037,765.500000,...,212.0,5300.0,0.227697,2.974891,2.747194,1.749945,1.047551,370.988392,1.930269,2.949403
3,15,86.0,2150.0,813.000000,830.000000,17.000000,820.588368,4.881966,70570.599670,819.700012,...,86.0,2150.0,0.011548,2.859551,2.848003,1.261054,0.846086,108.450662,1.058386,2.611284
4,17,61.0,1525.0,803.900024,812.299988,8.399963,806.972130,2.346612,49225.299927,806.799988,...,61.0,1525.0,0.065850,2.959481,2.893632,0.797028,0.866238,48.618682,0.355679,2.170918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12372,22201,37.0,925.0,936.200012,953.000000,16.799988,946.889187,3.692194,35034.899902,947.000000,...,37.0,925.0,0.007558,2.034825,2.027268,1.272405,0.724075,47.078981,1.655551,1.971356
12373,22202,35.0,875.0,758.900024,759.500000,0.599976,759.219999,0.173700,26572.699951,759.200012,...,35.0,875.0,2.105136,2.761521,0.656385,2.549493,0.159836,89.232252,2.573372,2.723249
12374,22203,26.0,650.0,863.099976,864.099976,1.000000,863.596154,0.283498,22453.500000,863.599976,...,26.0,650.0,3.241078,3.924465,0.683388,3.738636,0.187390,97.204536,3.802144,3.914294
12375,60319,83.0,2075.0,715.200012,729.700012,14.500000,721.809636,3.735491,59910.199829,721.400024,...,83.0,2075.0,0.004325,0.665635,0.661310,0.350760,0.182993,29.113115,0.395515,0.562083


In [75]:
result_df = pd.merge(point_df,df_merged,on=line_fields,how='outer')

In [76]:
result_df.to_csv(r'C:\Users\Runker\Desktop\CNT_TRAIN\train_table\pre.csv')

In [77]:
result_df

,OBJECTID_1,DEM_COUNT,DEM_AREA,DEM_MIN,DEM_MAX,DEM_RANGE,DEM_MEAN,DEM_STD,DEM_SUM,DEM_MEDIAN,...,MRVBF_COUNT,MRVBF_AREA,MRVBF_MIN,MRVBF_MAX,MRVBF_RANGE,MRVBF_MEAN,MRVBF_STD,MRVBF_SUM,MRVBF_MEDIAN,MRVBF_PCT90
0,6,27.0,675.0,1023.799988,1030.599976,6.799988,1027.181471,2.022101,27733.899719,1026.800049,...,27.0,675.0,0.009058,0.415623,0.406565,0.219172,0.118573,5.917642,0.226852,0.381662
1,7,40.0,1000.0,822.599976,834.400024,11.800049,826.194997,3.002248,33047.799866,825.600006,...,40.0,1000.0,0.001301,2.913924,2.912623,1.240179,1.043434,49.607156,1.272079,2.831860
2,8,212.0,5300.0,763.400024,771.000000,7.599976,765.955661,2.030737,162382.600037,765.500000,...,212.0,5300.0,0.227697,2.974891,2.747194,1.749945,1.047551,370.988392,1.930269,2.949403
3,15,86.0,2150.0,813.000000,830.000000,17.000000,820.588368,4.881966,70570.599670,819.700012,...,86.0,2150.0,0.011548,2.859551,2.848003,1.261054,0.846086,108.450662,1.058386,2.611284
4,17,61.0,1525.0,803.900024,812.299988,8.399963,806.972130,2.346612,49225.299927,806.799988,...,61.0,1525.0,0.065850,2.959481,2.893632,0.797028,0.866238,48.618682,0.355679,2.170918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12372,22201,37.0,925.0,936.200012,953.000000,16.799988,946.889187,3.692194,35034.899902,947.000000,...,37.0,925.0,0.007558,2.034825,2.027268,1.272405,0.724075,47.078981,1.655551,1.971356
12373,22202,35.0,875.0,758.900024,759.500000,0.599976,759.219999,0.173700,26572.699951,759.200012,...,35.0,875.0,2.105136,2.761521,0.656385,2.549493,0.159836,89.232252,2.573372,2.723249
12374,22203,26.0,650.0,863.099976,864.099976,1.000000,863.596154,0.283498,22453.500000,863.599976,...,26.0,650.0,3.241078,3.924465,0.683388,3.738636,0.187390,97.204536,3.802144,3.914294
12375,60319,83.0,2075.0,715.200012,729.700012,14.500000,721.809636,3.735491,59910.199829,721.400024,...,83.0,2075.0,0.004325,0.665635,0.661310,0.350760,0.182993,29.113115,0.395515,0.562083


In [16]:
polygon_path = polygon_path

In [19]:
# 添加字段
def add_field(input_table,field_name,field_type='DOUBLE'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)

In [4]:

# 新增面特征信息
# 添加新字段来存储四至坐标
for one_fields in ['XMin', 'YMin', 'XMax', 'YMax','Centroid_X','Centroid_Y']:
    add_field(polygon_path,one_fields)


In [5]:
# 更新字段值
with arcpy.da.UpdateCursor(polygon_path, ["SHAPE@", "XMin", "YMin", "XMax", "YMax", "Centroid_X", "Centroid_Y"]) as cursor:
    for row in cursor:
        centroid = row[0].centroid
        extent = row[0].extent
        row[1] = extent.XMin
        row[2] = extent.YMin
        row[3] = extent.XMax
        row[4] = extent.YMax
        row[5] = centroid.X
        row[6] = centroid.Y
        cursor.updateRow(row)

print("四至坐标计算完成并存储在新字段中。")


四至坐标计算完成并存储在新字段中。


In [20]:
polygon_fields_list = [_.name for _ in arcpy.ListFields(polygon_path)]

In [21]:
print(polygon_fields_list)

['FID', 'Shape', 'OBJECTID', 'DLBM', 'DLMC', 'Shape_Leng', 'Shape_Area']


In [12]:
polygon_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(polygon_path, ['FID','DLMC','new_TZ', 'XMin', 'YMin', 'XMax', 'YMax', 'Centroid_X', 'Centroid_Y','Riverdis']))

In [13]:
polygon_df['OBJECTID'] = polygon_df['FID']+1

In [14]:
polygon_df

,FID,DLMC,new_TZ,XMin,YMin,XMax,YMax,Centroid_X,Centroid_Y,Riverdis,OBJECTID
0,0,水田,黄浅白粉泥田,449006.3739,2.905394e+06,449046.2870,2.905463e+06,449025.945199,2.905434e+06,141.914994,1
1,1,水田,黄鳝泥田,455002.7485,2.910043e+06,455066.0906,2.910071e+06,455028.787173,2.910060e+06,654.616154,2
2,2,水田,黄浅白粉泥田,446745.1025,2.882598e+06,446800.9559,2.882628e+06,446776.231277,2.882614e+06,403.458976,3
3,3,水田,黄浅白粉泥田,446792.0015,2.882612e+06,446815.7014,2.882631e+06,446804.035531,2.882623e+06,423.481149,4
4,4,水田,浅石红泥田,463602.1111,2.901649e+06,463674.7285,2.901774e+06,463656.574150,2.901680e+06,0.000000,5
...,...,...,...,...,...,...,...,...,...,...,...
32140,32140,裸岩石砾地,薄层砾硅质黄壤,449535.0379,2.901705e+06,449596.5183,2.901769e+06,449563.836272,2.901735e+06,705.749931,32141
32141,32141,其他草地,腐薄层硅质黄壤,427064.7666,2.903328e+06,427085.3989,2.903347e+06,427072.791476,2.903339e+06,1363.427223,32142
32142,32142,其他林地,腐薄层红泥质黄壤,461597.6143,2.912454e+06,461670.2792,2.912491e+06,461633.377270,2.912471e+06,0.000000,32143
32143,32143,水田,石红泥田,458876.1215,2.905722e+06,458946.2619,2.905799e+06,458911.191700,2.905729e+06,1398.980051,32144


In [15]:
# 连接表格
result_df = pd.merge(polygon_df, df_merged, on='OBJECTID',how='left')

In [19]:
pd.unique(result_df['new_TZ']),len(pd.unique(result_df['new_TZ']))

(array(['黄浅白粉泥田', '黄鳝泥田', '浅石红泥田', '石红泥田', '潮砂泥田', '石灰泥田', '黄红泥田', '青石灰泥田',
        '黄黄砂泥田', '轻漂白粉泥田', '黄青白粉泥田', '黄浅红泥田', '黄白粉泥田', '黄浅砂泥田', '重漂红泥田',
        '潮泥田', '浅石灰泥田', '黄砂泥田', '黄浅鳝泥田', '重漂砂泥田', '薄层砾硅质黄壤', '厚层砂泥质黄壤',
        '中层灰泥质黄色石灰土', '厚层红泥质黄壤', '中层砂泥质黄壤', '薄层硅质酸性粗骨土', '薄层灰泥质黄色石灰土',
        '厚层黏质黄色石灰土', '薄层壤质中性紫色土', '薄层砾壤质黑色石灰土', '薄层砂泥质黄壤', '薄层砾壤质黄色石灰土',
        '薄层砾壤质中性紫色土', '腐厚层砂泥质黄壤', '腐薄层灰泥质黄壤', '腐中层砂泥质黄壤', '腐中层壤质黄色石灰土',
        '腐薄层硅质黄壤', '腐薄层硅质山地灌丛草甸土', '腐中层硅质黄壤', '腐薄层砂泥质黄壤', '腐薄层壤质黄色石灰土',
        '腐薄层砾砂质黑色石灰土', '腐厚层硅质黄壤', '腐中层壤质黑色石灰土', '腐厚层红泥质黄壤', '腐薄层黏质黄色石灰土',
        '腐厚层灰泥质黄壤', '腐中层红泥质黄壤', '腐中层灰泥质黄壤', '腐厚层壤质黄色石灰土', '腐中层砾壤质钙质紫色土',
        '腐中层砾壤质酸性紫色土', '腐薄层红泥质黄壤', '腐中层砂泥质黄壤性土', '腐厚层壤质黑色石灰土'],
       dtype=object),
 56)

In [17]:
# 保存训练数据
train_data_path = r'F:\cache_data\zone_ana\dy\train_data'
result_df.to_csv(os.path.join(train_data_path, 'result.csv'), index=False)

In [17]:
df_merged.to_csv(r'C:\Users\Runker\Desktop\river\river_table\river_polygon.csv')